In [3]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
import time

load_dotenv()

# Spotify API
CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')

/Users/brandonma/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
def read_and_combine_csv(folder_path):
    """
    Reads all CSV files in a folder and combines them into a single DataFrame.
    
    Parameters:
        folder_path (str): The path to the folder containing the CSV files.
        
    Returns:
        pd.DataFrame: A DataFrame containing the combined data from all CSV files.
    """
    # Initialize an empty list to store individual DataFrames
    dataframes = []
    
    # Iterate over all files in the folder
    for file_name in os.listdir(folder_path):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)
        
        # Check if the file is a CSV
        if file_name.endswith('.csv') and os.path.isfile(file_path):
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            dataframes.append(df)
    
    # Combine all DataFrames into one
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
    else:
        combined_df = pd.DataFrame()  # Return an empty DataFrame if no CSV files found
    
    return combined_df

In [6]:
# Access Token
def get_access_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }
    response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))
    response.raise_for_status()
    return response.json()["access_token"]

def get_audio_features(track_id, access_token):
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def get_audio_analysis(track_id, access_token):
    url = f"https://api.spotify.com/v1/audio-analysis/{track_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

## Data preparation

In [7]:
folder_path = "./data/track_history"
df = read_and_combine_csv(folder_path)

# Drop Duplicates
df = df.drop_duplicates()

# Modify the uri data
df["track_uri"] = df["track_uri"].str.replace("spotify:track:", "", regex=False)
df["artist_uri"] = df["artist_uri"].str.replace("spotify:artist:", "", regex=False)
df["album_uri"] = df["album_uri"].str.replace("spotify:album:", "", regex=False)

df.sample(10)
df.to_csv("./data/track_history.csv", index=False)

## Call API

In [ ]:
token = get_access_token(CLIENT_ID, CLIENT_SECRET)

track_ids = df['track_uri'].unique()

audio_features_list = []

def fetch_audio_features_batch(track_ids, token):
    url = "https://api.spotify.com/v1/audio-features"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers, params={"ids": ",".join(track_ids)})
    
    if response.status_code == 429:
        retry_after = response.headers.get("Retry-After", "1")
        print(f"Rate limited. Retry-After: {retry_after} seconds")
        print("Response headers:", response.headers)
        time.sleep(int(retry_after))
        return fetch_audio_features_batch(track_ids, token)

    response.raise_for_status()
    return response.json().get("audio_features", [])

for i in range(0, len(track_ids), 100):
    batch = track_ids[i:i+100]
    try:
        audio_features = fetch_audio_features_batch(batch, token)
        if audio_features:
            print(f"Successfully fetched audio features for batch starting with {batch[0]}")
        else:
            print(f"No audio features found for batch starting with {batch[0]}")
        audio_features_list.extend(audio_features)
        time.sleep(1)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching batch starting with {batch[0]}: {e}")

Successfully fetched audio features for batch starting with 5wbg8kepMFoMzHOEuxiI0q
Successfully fetched audio features for batch starting with 5rOkv636e3XHJmjLHvjLPT
Successfully fetched audio features for batch starting with 77x96TdWdkIia82QqU35SD
Successfully fetched audio features for batch starting with 6ShcaqMvlBcdsiZr23fDzk
Successfully fetched audio features for batch starting with 19Shlms2uTnOjIUg50TXzd
Successfully fetched audio features for batch starting with 2TX0eN7jdjQiGx7NHSax7l
Successfully fetched audio features for batch starting with 27U4sg4Z7tCnQ55jETIXDA
Successfully fetched audio features for batch starting with 0wPKDeY4fZXT6k9bzV0kx0
Successfully fetched audio features for batch starting with 1L3fMJNX5WUJ865uU406Vz
Successfully fetched audio features for batch starting with 7o3Zup0nTRFAFAPKBpS72G
Successfully fetched audio features for batch starting with 3Mgi95kwCC0jnYIx3Rrfi5


In [ ]:
import json

with open("./data/audio_features.json", "w") as json_file:
    json.dump(audio_features_list, json_file, indent=4)

## Clean audio features

In [ ]:
import pandas as pd

import json
with open("./data/audio_features.json", "r") as json_file:
    audio_features_list = json.load(json_file)

filtered_audio_features_list = [item for item in audio_features_list if item is not None and isinstance(item, dict)]

audio_features_df = pd.DataFrame(filtered_audio_features_list)

audio_features_df.to_csv("./data/audio_features.csv", index=False)

print("file saved")

file saved


In [12]:
columns_to_keep = ['id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
audio_features_df_filtered = audio_features_df[columns_to_keep]
audio_features_df_filtered.to_csv("./data/audio_features_filtered.csv", index=False)

## Combine Data

In [27]:
import pandas as pd
import json

# File paths
audio_features_path = "./data/audio_features_filtered.csv"
track_history_path = "./data/track_history.csv"

# Load CSV files
audio_features_df = pd.read_csv(audio_features_path)
track_history_df = pd.read_csv(track_history_path)

# Merge data
merged_df = track_history_df.merge(audio_features_df, left_on="track_uri", right_on="id", how="left")

# Rename the column for clarity
merged_df.rename(columns={"duration_ms_x": "duration_ms"}, inplace=True)

# Initialize JSON data list
result = []

# Group by track_id and construct JSON data
for track_id, group in merged_df.groupby("track_uri"):
    # Get the highest popularity
    highest_popularity = group["popularity"].max()
    
    # Get audio features (use the first row since they are consistent across the same track_id)
    audio_features = group.iloc[0][[
        "danceability", "energy", "key", "loudness", "mode",
        "speechiness", "acousticness", "instrumentalness",
        "liveness", "valence", "tempo", "time_signature"
    ]].to_dict()
    
    # Build listening history
    listening_history = group[["played_at", "context_type", "popularity"]].to_dict(orient="records")
    
    # Construct JSON data
    track_data = {
        "track_id": track_id,
        "track_name": group.iloc[0]["track_name"],
        "artist": {
            "artist_id": group.iloc[0]["artist_uri"],
            "artist_name": group.iloc[0]["artist_name"]
        },
        "album": {
            "album_id": group.iloc[0]["album_uri"],
            "album_name": group.iloc[0]["album_name"],
            "release_date": group.iloc[0]["release_date"],
            "total_tracks": int(group.iloc[0]["total_tracks"])
        },
        "duration_ms": int(group.iloc[0]["duration_ms"]),
        "isrc": group.iloc[0]["isrc"],
        "highest_popularity": int(highest_popularity),
        "audio_features": audio_features,
        "listening_history": listening_history
    }
    
    # Add to result list
    result.append(track_data)

# Save as JSON file
output_path = "./data/track_data.json"
with open(output_path, "w") as json_file:
    json.dump(result, json_file, indent=4)

print(f"Data successfully saved to {output_path}")

Data successfully saved to ./data/track_data.json
